In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from PIL import Image
import os
import io
import glob
import clip  # OpenAI's CLIP package
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np

import numpy as np
from PIL import Image
import glob
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms,models
import torch.nn as nn
import torch
from PIL import Image
import glob
import torch
import clip
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, Dataset
import cv2


In [2]:
# Load the CLIP model
gpu_id = 2  # Change this to select a different GPU
device = f"cuda:{gpu_id}" if torch.cuda.is_available() else "cpu"

# Define transformations (data augmentation + normalization)
def get_transforms():
    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Resize to ResNet-50 input size
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    return transform

class CLIPDataset(Dataset):
    def __init__(self, image_paths, labels, transform, USE_DFT, USE_CROSS_DIFF):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform
        self.USE_DFT = USE_DFT
        self.USE_CROSS_DIFF = USE_CROSS_DIFF

    def __len__(self):
        return len(self.image_paths)

    
    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert("RGB")
        image_np = np.array(image)

        if self.USE_DFT:
            image_np = np.array(image)
            dft = np.fft.fft2(image_np, axes=(0, 1))
            dft_shifted = np.fft.fftshift(dft)
            magnitude_spectrum = 20 * np.log(np.abs(dft_shifted) + 1)
            image_np = np.uint8(np.clip(magnitude_spectrum, 0, 255))
            image = Image.fromarray(image_np.astype(np.uint8))

        if self.USE_CROSS_DIFF:
            image_np = np.array(image)
            blurred = cv2.GaussianBlur(image_np, (3, 3), 0)
            diff = cv2.absdiff(image_np, blurred)
            image = Image.fromarray(diff.astype(np.uint8))

        image = self.transform(image)
        label = self.labels[idx]
        return image, label

# Define the file paths for training set
def get_dataset(transform, USE_DFT, USE_CROSS_DIFF):
    file_paths = [
        '/data/home/saisuchithm/godwin/mlsp/project/diffusion_model_deepfakes_lsun_bedroom/train/ADM/1_fake/*.png',
        '/data/home/saisuchithm/godwin/mlsp/project/diffusion_model_deepfakes_lsun_bedroom/train/DDPM/1_fake/*.png',
        '/data/home/saisuchithm/godwin/mlsp/project/diffusion_model_deepfakes_lsun_bedroom/train/Diff-ProjectedGAN/1_fake/*.png',
        '/data/home/saisuchithm/godwin/mlsp/project/diffusion_model_deepfakes_lsun_bedroom/train/Diff-StyleGAN2/1_fake/*.png',
        '/data/home/saisuchithm/godwin/mlsp/project/diffusion_model_deepfakes_lsun_bedroom/train/IDDPM/1_fake/*.png',
        '/data/home/saisuchithm/godwin/mlsp/project/diffusion_model_deepfakes_lsun_bedroom/train/LDM/1_fake/*.png',
        '/data/home/saisuchithm/godwin/mlsp/project/diffusion_model_deepfakes_lsun_bedroom/train/PNDM/1_fake/*.png',
        '/data/home/saisuchithm/godwin/mlsp/project/diffusion_model_deepfakes_lsun_bedroom/train/ProGAN/1_fake/*.png',
        '/data/home/saisuchithm/godwin/mlsp/project/diffusion_model_deepfakes_lsun_bedroom/train/ProjectedGAN/1_fake/*.png',
        '/data/home/saisuchithm/godwin/mlsp/project/diffusion_model_deepfakes_lsun_bedroom/train/StyleGAN/1_fake/*.png',
        '/data/home/saisuchithm/godwin/mlsp/project/diffusion_model_deepfakes_lsun_bedroom/train/Real/0_real/*.jpg'
    ]

    label_ADM = 0
    label_DDPM = 1
    label_Diff_Pro_GAN = 2
    label_Diff_Style_GAN = 3
    label_IDDPM = 4
    label_LDM = 5
    label_PNDM = 6
    label_ProGAN = 7
    label_ProjectedGAN = 8
    label_StyleGAN = 9
    label_Real = 10

    # List of labels
    labels = [label_ADM, label_DDPM, label_Diff_Pro_GAN, label_Diff_Style_GAN, label_IDDPM, label_LDM, label_PNDM, label_ProGAN, label_ProjectedGAN, label_StyleGAN, label_Real]


    # Collect all image paths and corresponding labels
    image_paths = []
    image_labels = []

    for path, label in zip(file_paths, labels):
        images = glob.glob(path)  # Get all image file paths in the folder
        image_paths.extend(images)
        image_labels.extend([label] * len(images))  # Assign the same label to all images in that folder

    # Check if images and labels are aligned
    assert len(image_paths) == len(image_labels)

    # Example usage
    dataset = CLIPDataset(image_paths, image_labels, transform, USE_DFT, USE_CROSS_DIFF)

    # Define the file paths for validation set
    # Define the file paths for training set
    val_file_paths = [
        '/data/home/saisuchithm/godwin/mlsp/project/diffusion_model_deepfakes_lsun_bedroom/val/ADM/1_fake/*.png',
        '/data/home/saisuchithm/godwin/mlsp/project/diffusion_model_deepfakes_lsun_bedroom/val/DDPM/1_fake/*.png',
        '/data/home/saisuchithm/godwin/mlsp/project/diffusion_model_deepfakes_lsun_bedroom/val/Diff-ProjectedGAN/1_fake/*.png',
        '/data/home/saisuchithm/godwin/mlsp/project/diffusion_model_deepfakes_lsun_bedroom/val/Diff-StyleGAN2/1_fake/*.png',
        '/data/home/saisuchithm/godwin/mlsp/project/diffusion_model_deepfakes_lsun_bedroom/val/IDDPM/1_fake/*.png',
        '/data/home/saisuchithm/godwin/mlsp/project/diffusion_model_deepfakes_lsun_bedroom/val/LDM/1_fake/*.png',
        '/data/home/saisuchithm/godwin/mlsp/project/diffusion_model_deepfakes_lsun_bedroom/val/PNDM/1_fake/*.png',
        '/data/home/saisuchithm/godwin/mlsp/project/diffusion_model_deepfakes_lsun_bedroom/val/ProGAN/1_fake/*.png',
        '/data/home/saisuchithm/godwin/mlsp/project/diffusion_model_deepfakes_lsun_bedroom/val/ProjectedGAN/1_fake/*.png',
        '/data/home/saisuchithm/godwin/mlsp/project/diffusion_model_deepfakes_lsun_bedroom/val/StyleGAN/1_fake/*.png',
        '/data/home/saisuchithm/godwin/mlsp/project/diffusion_model_deepfakes_lsun_bedroom/val/Real/0_real/*.jpg'
    ]

    label_ADM = 0
    label_DDPM = 1
    label_Diff_Pro_GAN = 2
    label_Diff_Style_GAN = 3
    label_IDDPM = 4
    label_LDM = 5
    label_PNDM = 6
    label_ProGAN = 7
    label_ProjectedGAN = 8
    label_StyleGAN = 9
    label_Real = 10

    # List of labels
    val_labels = [label_ADM, label_DDPM, label_Diff_Pro_GAN, label_Diff_Style_GAN, label_IDDPM, label_LDM, label_PNDM, label_ProGAN, label_ProjectedGAN, label_StyleGAN, label_Real]

    # Collect all image paths and corresponding labels
    val_image_paths = []
    val_image_labels = []

    for path, label in zip(val_file_paths, val_labels):
        images = glob.glob(path)  # Get all image file paths in the folder
        val_image_paths.extend(images)
        val_image_labels.extend([label] * len(images))

    # Check if images and labels are aligned
    assert len(val_image_paths) == len(val_image_labels)

    val_dataset = CLIPDataset(val_image_paths, val_image_labels, transform, USE_DFT, USE_CROSS_DIFF)

    return dataset, val_dataset

# Load pretrained ResNet-50 model
class ResNet50MultiClassClassifier(nn.Module):
    def __init__(self, pretrained=True, num_classes=11):
        super(ResNet50MultiClassClassifier, self).__init__()
        self.model = models.resnet50(pretrained=pretrained)
        num_ftrs = self.model.fc.in_features
        self.model.fc = nn.Linear(num_ftrs, num_classes)  # 11 classes (10 AI-generated types + 1 real image)

    def forward(self, x):
        return self.model(x)
    

# Simple linear head for classification
class CLIPClassifier(nn.Module):
    def __init__(self, clip_model, num_classes=11):
        super().__init__()
        self.clip_model = clip_model
        self.classifier = nn.Sequential(
                            nn.LayerNorm(512),
                            nn.Linear(512, num_classes)
                            )

    def forward(self, x):
        with torch.no_grad():
            features = self.clip_model.encode_image(x)
        features = features.float()
        return self.classifier(features)
    
class EarlyStopping:
    def __init__(self, model, patience=5, min_delta=0.001):
        """
        Args:
            patience (int): Number of epochs to wait after last improvement.
            min_delta (float): Minimum change to qualify as an improvement.
        """
        self.patience = patience
        self.min_delta = min_delta
        self.best_loss = float("inf")
        self.counter = 0
        self.early_stop = False
        self.model = model

    def __call__(self, val_loss, model):
        if val_loss < self.best_loss - self.min_delta:
            self.best_loss = val_loss
            self.counter = 0  # Reset counter if loss improves
            self.model = model
        else:
            self.counter += 1  # Increase counter if no improvement
            if self.counter >= self.patience:
                self.early_stop = True

#Ensemble of the CNN model and CLIP Model
class EnsembleModel(nn.Module):
    def __init__(self, cnn_model, cnn_dft_model, clip_model):
        super(EnsembleModel, self).__init__()
        self.cnn_model = cnn_model
        self.cnn_dft_model = cnn_dft_model
        self.clip_model = clip_model

        self.weight = nn.Parameter(torch.tensor([1.0, 1.0, 1.0]))
        self.softmax = nn.Softmax()
        #Freeze the CNN and CLIP parameters
        for param in self.cnn_model.parameters():
            param.requires_grad = False
        for param in self.cnn_dft_model.parameters():
            param.requires_grad = False
        for param in self.clip_model.parameters():
            param.requires_grad = False
    
    def preprocess_CNN(self, image):
        image_np = image.cpu().numpy()  # (32, 3, 224, 224)
        
        processed_images = []

        for img in image_np:  # loop over 32 images
            img = np.transpose(img, (1, 2, 0))  # (224, 224, 3)

            dft = np.fft.fft2(img, axes=(0, 1))
            dft_shifted = np.fft.fftshift(dft)
            magnitude_spectrum = 20 * np.log(np.abs(dft_shifted) + 1)
            magnitude_spectrum = np.clip(magnitude_spectrum, 0, 255).astype(np.uint8)

            magnitude_spectrum = np.transpose(magnitude_spectrum, (2, 0, 1))  # back to (3, 224, 224)
            processed_images.append(torch.from_numpy(magnitude_spectrum))

        image = torch.stack(processed_images).float().to(device)  # (32, 3, 224, 224)

        return image

    def forward(self, x):
      #Preprocess the image since CNN was trained on DFTed Images
      cnn_image = self.preprocess_CNN(x)
      #Pass the dfted image through the CNN Model
      cnn_dft_features = self.cnn_dft_model(cnn_image)
      #Pass the image through CNN model
      cnn_features = self.cnn_model(x)
      #Pass the image though the CLIP model
      clip_features = self.clip_model(x)
      #Linear combination of prediction of the two models
      normalized_weight = self.softmax(self.weight)
      output = normalized_weight[0] * cnn_features + normalized_weight[1] * clip_features + normalized_weight[2] * cnn_dft_features
      return output

    
# Training Loop
def train_ensemble(model, train_dummy_loader, val_dummy_loader, criterion, optimizer, num_epochs, breakpoint = 2000):
    early_stopping = EarlyStopping(model,patience=3, min_delta=0.001)
    training_loss_list = []
    training_accuracy_list = []
    val_loss_list = []
    val_accuracy_list = []
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        total_accuracy = 0
        for batch_idx, (images, labels) in enumerate(train_dummy_loader):
            images, labels = images.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            predicted = torch.argmax(outputs, dim=1)
            #Calcualte the accuracy
            accuracy = (predicted == labels).sum().item() / len(labels)
            total_accuracy += accuracy
            
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
            print(f"Epoch [{epoch+1}/{num_epochs}], Batch [{batch_idx+1}/{len(train_dummy_loader)}], Loss: {loss.item():.4f}, Accuracy: {accuracy:.4f}", end='\r')
            if batch_idx > breakpoint:
                break
        avg_loss = total_loss /( batch_idx+1)
        avg_accuracy = total_accuracy / (batch_idx+1)
        training_loss_list.append(loss.item())
        training_accuracy_list.append(accuracy)
        print(f"Epoch {epoch+1}/{num_epochs}, Average Loss: {avg_loss:.4f}, Average Accuracy: {avg_accuracy:.4f}", end='\r')

        # Validation 
        model.eval()
        with torch.no_grad():
            total_loss = 0.0
            total_accuracy = 0.0
            for val_idx, (images, labels) in enumerate(val_dummy_loader):
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                total_loss += loss.item()
                predicted = torch.argmax(outputs, dim=1)
                #Calcualte the accuracy
                accuracy = (predicted == labels).sum().item() / len(labels)
                total_accuracy += accuracy
                if val_idx > breakpoint:
                    break
            avg_loss = total_loss / (val_idx+1)
            avg_accuracy = total_accuracy / (val_idx+1)
            val_loss_list.append(avg_loss)
            val_accuracy_list.append(avg_accuracy)
            early_stopping(avg_loss, model)
            print(f"\nValidation Loss: {avg_loss:.4f}, Accuracy: {avg_accuracy:.4f}\n", end='\r')
            
            if early_stopping.early_stop:
                print("Early stopping triggered!")
                break
    return training_accuracy_list, training_loss_list, val_loss_list, val_accuracy_list, early_stopping.model


In [3]:
model_clip, preprocess_clip = clip.load("ViT-B/32", device=device)

dataset, val_dataset = get_dataset(transform = preprocess_clip, USE_DFT=False, USE_CROSS_DIFF=False)

#Make the dataloaders
train_dataloader = DataLoader(dataset, batch_size=32, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False)

#Initialize the clip model with the trained weights
clip_classifier = CLIPClassifier(model_clip).to(device)
clip_classifier.load_state_dict(torch.load("/data/home/saisuchithm/godwin/mlsp/project/Trained_CLIP.pth", weights_only=True))
clip_classifier.eval()

#Initialize the Resnet model with trained weights
cnn_model = ResNet50MultiClassClassifier().to(device)
cnn_model.load_state_dict(torch.load("/data/home/saisuchithm/godwin/mlsp/project/MLSP_CNN.pth", weights_only=True))
cnn_model.eval()

#Initialize the CNN model with DFT with trained weights
cnn_dft_model = ResNet50MultiClassClassifier().to(device)
cnn_dft_model.load_state_dict(torch.load("/data/home/saisuchithm/godwin/mlsp/project/MLSP_CNN_DFT.pth", weights_only=True))
cnn_dft_model.eval()


#Initialize the ensemble model
ensemble_model = EnsembleModel(cnn_model, cnn_dft_model, clip_classifier).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(ensemble_model.parameters(), lr=1e-4, weight_decay = 1e-5)


/data/home/saisuchithm/python-envs/ai-gen-detect/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/data/home/saisuchithm/python-envs/ai-gen-detect/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [4]:
# training_accuracy_list, training_loss_list, val_loss_list, val_accuracy_list, model_trainined = train_ensemble(model=ensemble_model,
#                                                                                                     train_dummy_loader=train_dataloader,
#                                                                                                     val_dummy_loader=val_dataloader,
#                                                                                                     criterion=criterion,
#                                                                                                     optimizer=optimizer,
#                                                                                                     num_epochs=1)

In [5]:
label_idx_to_name = {
    0: "Picture of AI generated image, from ADM method",
    1: "Picture of AI generated image, from DDPM method",
    2: "Picture of AI generated image, from Diff-ProjectedGAN method",
    3: "Picture of AI generated image, from Diff-StyleGAN",
    4: "Picture of AI generated image, from IDDPM method",
    5: "Picture of AI generated image, from LDM method",
    6: "Picture of AI generated image, from PNDM method",
    7: "Picture of AI generated image, from ProGAN method",
    8: "Picture of AI generated image, from ProjectedGAN method",
    9: "Picture of AI generated image, from StyleGAN method",
    10: "Picture of a Real Image"
}

In [6]:
ensemble_model.load_state_dict(torch.load("/data/home/saisuchithm/godwin/mlsp/project/Ensemble.pth", weights_only=True, map_location=device))
ensemble_model.eval()

EnsembleModel(
  (cnn_model): ResNet50MultiClassClassifier(
    (model): ResNet(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    

In [18]:
accuracy_class = []
breakpoint = 500
for i in range(11):
    first_index = i*1000
    last_index = (i+1)*1000
    accuracy = 0
    print(f"Running for {label_idx_to_name.get(i)}")
    count = 0
    for j in range(first_index, last_index):
      image = val_dataset[j][0]
      label = val_dataset[j][1]
      image = image.unsqueeze(0)
      image = image.to(device)
      label = torch.tensor([label]).to(device)
      output = ensemble_model(image)
      predicted = torch.argmax(output, dim=1)
      if predicted == label:
        accuracy += 1
      count += 1
      if count == breakpoint:
         break
    accuracy_class.append(accuracy/breakpoint)
print(accuracy_class)

Running for Picture of AI generated image, from ADM method
Running for Picture of AI generated image, from DDPM method
Running for Picture of AI generated image, from Diff-ProjectedGAN method
Running for Picture of AI generated image, from Diff-StyleGAN
Running for Picture of AI generated image, from IDDPM method
Running for Picture of AI generated image, from LDM method
Running for Picture of AI generated image, from PNDM method
Running for Picture of AI generated image, from ProGAN method
Running for Picture of AI generated image, from ProjectedGAN method
Running for Picture of AI generated image, from StyleGAN method
Running for Picture of a Real Image
[0.846, 1.0, 0.992, 0.998, 0.936, 1.0, 0.998, 0.998, 0.996, 0.998, 0.866]


In [25]:
label_idx_to_name = {
    0: "Picture of AI generated image, from ADM method",
    1: "Picture of AI generated image, from DDPM method",
    2: "Picture of AI generated image, from Diff-ProjectedGAN method",
    3: "Picture of AI generated image, from Diff-StyleGAN",
    4: "Picture of AI generated image, from IDDPM method",
    5: "Picture of AI generated image, from LDM method",
    6: "Picture of AI generated image, from PNDM method",
    7: "Picture of AI generated image, from ProGAN method",
    8: "Picture of AI generated image, from ProjectedGAN method",
    9: "Picture of AI generated image, from StyleGAN method",
    10: "Picture of a Real Image"
}
arr = accuracy_class
print("Results of Ensemble Model")
for i in range(len(arr)):
    print(f"Accuracy for {label_idx_to_name.get(i)} : {arr[i]}")

Results of Ensemble Model
Accuracy for Picture of AI generated image, from ADM method : 0.946
Accuracy for Picture of AI generated image, from DDPM method : 1.0
Accuracy for Picture of AI generated image, from Diff-ProjectedGAN method : 0.992
Accuracy for Picture of AI generated image, from Diff-StyleGAN : 0.998
Accuracy for Picture of AI generated image, from IDDPM method : 0.936
Accuracy for Picture of AI generated image, from LDM method : 1.0
Accuracy for Picture of AI generated image, from PNDM method : 0.998
Accuracy for Picture of AI generated image, from ProGAN method : 0.998
Accuracy for Picture of AI generated image, from ProjectedGAN method : 0.996
Accuracy for Picture of AI generated image, from StyleGAN method : 0.998
Accuracy for Picture of a Real Image : 0.966


### Testing on a Model as a AI vs Real Classifier


In [6]:
ensemble_model.load_state_dict(torch.load("/data/home/saisuchithm/godwin/mlsp/project/Ensemble.pth", weights_only=True, map_location=device))
ensemble_model.eval()

EnsembleModel(
  (cnn_model): ResNet50MultiClassClassifier(
    (model): ResNet(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    

In [7]:
dataset, val_dataset = get_dataset(transform = preprocess_clip, USE_DFT=False, USE_CROSS_DIFF=False)

#Make the dataloaders
val_dataloader = DataLoader(val_dataset, batch_size=128, shuffle=True)

In [53]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
y_true = []
y_pred = []

with torch.no_grad():
    for idx, (images, labels) in enumerate(val_dataloader):
        print(f"Running for the batch {idx + 1}")
        images = images.to(device)
        outputs = ensemble_model(images)

        _, predicted = torch.max(outputs, 1)

        # Convert predicted classes (0–10) to binary: AI (0–9) => 0, Real (10) => 1
        binary_preds = (predicted == 10).long()
        y_pred.extend(binary_preds.cpu().tolist())
        print(labels.max(), labels.min())
        true_labels = [1 if label == 10 else 0 for label in labels]
        y_true.extend(true_labels)

# --- Evaluation ---

print("Accuracy:", accuracy_score(y_true, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_true, y_pred))
print("Classification Report:\n", classification_report(y_true, y_pred, target_names=["AI", "Real"]))

Running for the batch 1


/data/home/saisuchithm/python-envs/ai-gen-detect/lib/python3.8/site-packages/torch/nn/modules/module.py:1553: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


tensor(10) tensor(0)
Running for the batch 2
tensor(10) tensor(0)
Running for the batch 3
tensor(10) tensor(0)
Running for the batch 4
tensor(10) tensor(0)
Running for the batch 5
tensor(10) tensor(0)
Running for the batch 6
tensor(10) tensor(0)
Running for the batch 7
tensor(10) tensor(0)
Running for the batch 8
tensor(10) tensor(0)
Running for the batch 9
tensor(10) tensor(0)
Running for the batch 10
tensor(10) tensor(1)
Running for the batch 11
tensor(10) tensor(0)
Running for the batch 12
tensor(10) tensor(0)
Running for the batch 13
tensor(10) tensor(0)
Running for the batch 14
tensor(10) tensor(0)
Running for the batch 15
tensor(10) tensor(0)
Running for the batch 16
tensor(10) tensor(0)
Running for the batch 17
tensor(10) tensor(0)
Running for the batch 18
tensor(10) tensor(0)
Running for the batch 19
tensor(10) tensor(0)
Running for the batch 20
tensor(10) tensor(0)
Running for the batch 21
tensor(10) tensor(0)
Running for the batch 22
tensor(10) tensor(0)
Running for the batch

#  Replay-Based Incremental Learning

In [7]:
import os
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

class RealVsAIDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.samples = []
        self.transform = transform
        self.label_map = {"AI": 0, "Real": 1}

        for class_dir in os.listdir(root_dir):
            class_path = os.path.join(root_dir, class_dir)
            if not os.path.isdir(class_path):
                continue

            # Determine if it's AI or Real based on folder name
            label = 0 if class_dir.startswith("AI_") else 1

            # Collect image paths and their binary label
            for fname in os.listdir(class_path):
                fpath = os.path.join(class_path, fname)
                if os.path.isfile(fpath) and fname.lower().endswith((".png", ".jpg", ".jpeg")):
                    self.samples.append((fpath, label))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        image_path, label = self.samples[idx]
        image = Image.open(image_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        return image, label

# Example transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)
])



In [8]:
class OldDatasetBinaryWrapper(Dataset):
    def __init__(self, original_dataset):
        self.original_dataset = original_dataset

    def __len__(self):
        return len(self.original_dataset)

    def __getitem__(self, idx):
        image, label = self.original_dataset[idx]
        binary_label = 0 if label < 10 else 1  # 0–9 → AI, 10 → Real
        return image, binary_label


In [14]:
class RealVsAIDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.samples = []
        self.transform = transform
        self.labels = []
        for class_dir in os.listdir(root_dir):
            class_path = os.path.join(root_dir, class_dir)
            if not os.path.isdir(class_path):
                continue

            label = 0 if class_dir.startswith("AI_") else 1  # 0 = AI, 1 = Real

            for fname in os.listdir(class_path):
                fpath = os.path.join(class_path, fname)
                if os.path.isfile(fpath) and fname.lower().endswith((".png", ".jpg", ".jpeg")):
                    self.samples.append((fpath, label))
                    self.labels.append(label)
    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        image_path, label = self.samples[idx]
        image = Image.open(image_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, label


In [10]:
# Simple linear head for classification
class AI_Detector(nn.Module):
    def __init__(self, ensemble_model):
        super().__init__()
        self.ensemble_model = ensemble_model
        self.classifier = nn.Sequential(
                            nn.LayerNorm(11),
                            nn.Linear(11, 64), 
                            nn.ReLU(), 
                            nn.Linear(64, 1)
                            )
        #Freeze all of the ensemble parameters
        for param in self.ensemble_model.parameters():
            param.requires_grad = False

    def forward(self, x):
        with torch.no_grad():
            features = self.ensemble_model(x)
        features = features.float()
        return self.classifier(features)

In [15]:
from torch.utils.data import ConcatDataset, DataLoader, Subset
import random

# Transforms (same for both datasets)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)
])

# Old dataset (assuming you already have it as `old_dataset`)
old_dataset_binary_train = OldDatasetBinaryWrapper(dataset)
old_dataset_binary_valid= OldDatasetBinaryWrapper(val_dataset)

# Assume old_dataset_binary is the binary-wrapped version of old dataset
subset_size_train = int(0.2 * len(old_dataset_binary_train))  # use 20%
subset_size_val = int(0.2 * len(old_dataset_binary_valid))  # use 20%

indices_train = random.sample(range(len(old_dataset_binary_train)), subset_size_train)
indices_val = random.sample(range(len(old_dataset_binary_valid)), subset_size_val)

old_subset_train = Subset(old_dataset_binary_train, indices_train)
old_subset_val = Subset(old_dataset_binary_valid, indices_val)

# New dataset from folder
new_dataset_train = RealVsAIDataset("/data/home/saisuchithm/godwin/mlsp/project/AI-Artwork/Real_AI_SD_LD_Dataset/train", transform=transform)
new_dataset_val = RealVsAIDataset("/data/home/saisuchithm/godwin/mlsp/project/AI-Artwork/Real_AI_SD_LD_Dataset/test", transform=transform)

# Combine
combined_dataset_train = ConcatDataset([old_subset_train, new_dataset_train])
combined_dataset_val = ConcatDataset([old_subset_val, new_dataset_val])
train_loader = DataLoader(combined_dataset_train, batch_size=64, shuffle=True)
val_loader = DataLoader(combined_dataset_val, batch_size=64, shuffle=True)

In [13]:
ensemble_model.load_state_dict(torch.load("/data/home/saisuchithm/godwin/mlsp/project/Ensemble.pth", weights_only=True, map_location=device))
ensemble_model.eval()

ai_detector = AI_Detector(ensemble_model=ensemble_model).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(ai_detector.parameters(), lr=1e-4, weight_decay=1e-5)

In [ ]:
import os
import torch
import numpy as np

# Setup
num_epochs = 5
train_break_point = 1000
checkpoint_dir = "/data/home/saisuchithm/godwin/mlsp/project/checkpoints"
os.makedirs(checkpoint_dir, exist_ok=True)

best_val_accuracy = 0.0

for epoch in range(num_epochs):
    ai_detector.train()
    running_loss = []
    running_accuracy = []

    for idx, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.float().to(device)  # BCE loss expects float labels

        logits = ai_detector(images).squeeze(1)
        loss = criterion(logits, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss.append(loss.item())

        # Binary prediction
        predicted = (torch.sigmoid(logits) > 0.5).float()
        accuracy = (predicted == labels).sum().item() / len(labels)
        running_accuracy.append(accuracy)

        print(f"Epoch: {epoch + 1}/{num_epochs} Batch: {idx + 1}/{len(train_loader)} loss: {loss.item():.4f} accuracy: {accuracy:.4f}", end="\r")

        if idx > train_break_point:
            break

    train_loss = np.mean(running_loss)
    train_acc = np.mean(running_accuracy)
    print(f"\nEpoch {epoch+1}/{num_epochs} - Training Loss: {train_loss:.4f}, Training Accuracy: {train_acc:.4f}")

    # ---- Validation ----
    ai_detector.eval()
    val_running_loss = []
    val_running_accuracy = []

    with torch.no_grad():
        for val_idx, (val_images, val_labels) in enumerate(val_loader):
            val_images = val_images.to(device)
            val_labels = val_labels.float().to(device)

            logits_val = ai_detector(val_images).squeeze(1)
            loss_val = criterion(logits_val, val_labels)
            val_running_loss.append(loss_val.item())

            predicted_val = (torch.sigmoid(logits_val) > 0.5).float()
            val_acc = (predicted_val == val_labels).sum().item() / len(val_labels)
            val_running_accuracy.append(val_acc)

    val_loss = np.mean(val_running_loss)
    val_acc = np.mean(val_running_accuracy)
    print(f"Epoch {epoch+1}/{num_epochs} - Validation Loss: {val_loss:.4f} - Validation Accuracy: {val_acc:.4f}")

    # ---- Save checkpoint ----
    checkpoint_path = os.path.join(checkpoint_dir, f"epoch_{epoch+1}.pt")
    torch.save({
        "epoch": epoch + 1,
        "model_state_dict": ai_detector.state_dict(),
        "optimizer_state_dict": optimizer.state_dict(),
        "val_accuracy": val_acc
    }, checkpoint_path)

    # Optionally save best model
    if val_acc > best_val_accuracy:
        best_val_accuracy = val_acc
        torch.save(ai_detector.state_dict(), os.path.join(checkpoint_dir, "best_model.pt"))


Epoch: 1/5 Batch: 1002/3763 loss: 0.5087 accuracy: 0.7812
Epoch 1/5 - Training Loss: 0.4889, Training Accuracy: 0.7750
Epoch 1/5 - Validation Loss: 0.4684 - Validation Accuracy: 0.8002
Epoch: 2/5 Batch: 1002/3763 loss: 0.3282 accuracy: 0.9062
Epoch 2/5 - Training Loss: 0.3621, Training Accuracy: 0.8581
Epoch 2/5 - Validation Loss: 0.3840 - Validation Accuracy: 0.8562
Epoch: 3/5 Batch: 1002/3763 loss: 0.3833 accuracy: 0.8438
Epoch 3/5 - Training Loss: 0.3178, Training Accuracy: 0.8677
Epoch 3/5 - Validation Loss: 0.3368 - Validation Accuracy: 0.8699
Epoch: 4/5 Batch: 1002/3763 loss: 0.3480 accuracy: 0.8281
Epoch 4/5 - Training Loss: 0.2868, Training Accuracy: 0.8751
Epoch 4/5 - Validation Loss: 0.3106 - Validation Accuracy: 0.8777
Epoch: 5/5 Batch: 1002/3763 loss: 0.2545 accuracy: 0.8594
Epoch 5/5 - Training Loss: 0.2711, Training Accuracy: 0.8828
Epoch 5/5 - Validation Loss: 0.2970 - Validation Accuracy: 0.8826


In [65]:
torch.save(ai_detector.state_dict(), "ai_detector.pth")
print("Model saved successfully!")

Model saved successfully!


In [ ]:
#Resume from a checkpoint:
checkpoint = torch.load("./checkpoints/epoch_3.pt")
ai_detector.load_state_dict(checkpoint["model_state_dict"])
optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
start_epoch = checkpoint["epoch"]

#### Testing the AI-Classifier on the Test Data

In [ ]:
ai_detector.load_state_dict(torch.load("/data/home/saisuchithm/godwin/mlsp/project/ai_detector.pth", weights_only=True, map_location=device))
ai_detector.eval()

In [26]:
dataset, val_dataset = get_dataset(transform = preprocess_clip, USE_DFT=False, USE_CROSS_DIFF=False)

#Make the dataloaders
old_dataset_binary_valid= OldDatasetBinaryWrapper(val_dataset)
val_dataloader = DataLoader(old_dataset_binary_valid, batch_size=50, shuffle=True)

In [41]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
y_true = []
y_pred = []
ai_detector.eval()

with torch.no_grad():
    for idx, (images, labels) in enumerate(val_dataloader):
        print(f"Running for the batch {idx + 1}")
        images = images.to(device)
        outputs = ai_detector(images)

        predicted = (torch.sigmoid(outputs) > 0.5).long()
        y_pred.extend(predicted.cpu().tolist())
        y_true.extend(labels.cpu().tolist())

# --- Evaluation ---
print(y_true)
print(y_pred)
print("Accuracy:", accuracy_score(y_true, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_true, y_pred))
print("Classification Report:\n", classification_report(y_true, y_pred, target_names=["AI", "Real"]))

Running for the batch 1


/data/home/saisuchithm/python-envs/ai-gen-detect/lib/python3.8/site-packages/torch/nn/modules/module.py:1553: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


Running for the batch 2
Running for the batch 3
Running for the batch 4
Running for the batch 5
Running for the batch 6
Running for the batch 7
Running for the batch 8
Running for the batch 9
Running for the batch 10
Running for the batch 11
Running for the batch 12
Running for the batch 13
Running for the batch 14
Running for the batch 15
Running for the batch 16
Running for the batch 17
Running for the batch 18
Running for the batch 19
Running for the batch 20
Running for the batch 21
Running for the batch 22
Running for the batch 23
Running for the batch 24
Running for the batch 25
Running for the batch 26
Running for the batch 27
Running for the batch 28
Running for the batch 29
Running for the batch 30
Running for the batch 31
Running for the batch 32
Running for the batch 33
Running for the batch 34
Running for the batch 35
Running for the batch 36
Running for the batch 37
Running for the batch 38
Running for the batch 39
Running for the batch 40
Running for the batch 41
Running 

In [43]:
new_dataset_val = RealVsAIDataset("/data/home/saisuchithm/godwin/mlsp/project/AI-Artwork/Real_AI_SD_LD_Dataset/test", transform=transform)
new_val_dataloader = DataLoader(new_dataset_val, batch_size=128, shuffle=True)

In [45]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
y_true = []
y_pred = []
ai_detector.eval()

with torch.no_grad():
    for idx, (images, labels) in enumerate(new_val_dataloader):
        print(f"Running for the batch {idx + 1}")
        images = images.to(device)
        outputs = ai_detector(images)

        predicted = (torch.sigmoid(outputs) > 0.5).long()
        y_pred.extend(predicted.cpu().tolist())
        y_true.extend(labels.cpu().tolist())

# --- Evaluation ---

print("Accuracy:", accuracy_score(y_true, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_true, y_pred))
print("Classification Report:\n", classification_report(y_true, y_pred, target_names=["AI", "Real"]))

Running for the batch 1


/data/home/saisuchithm/python-envs/ai-gen-detect/lib/python3.8/site-packages/torch/nn/modules/module.py:1553: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


Running for the batch 2
Running for the batch 3
Running for the batch 4
Running for the batch 5
Running for the batch 6
Running for the batch 7
Running for the batch 8
Running for the batch 9
Running for the batch 10
Running for the batch 11
Running for the batch 12
Running for the batch 13
Running for the batch 14
Running for the batch 15
Running for the batch 16
Running for the batch 17
Running for the batch 18
Running for the batch 19
Running for the batch 20
Running for the batch 21
Running for the batch 22
Running for the batch 23
Running for the batch 24
Running for the batch 25
Running for the batch 26
Running for the batch 27
Running for the batch 28
Running for the batch 29
Running for the batch 30
Running for the batch 31
Running for the batch 32
Running for the batch 33
Running for the batch 34
Running for the batch 35
Running for the batch 36
Running for the batch 37
Running for the batch 38
Running for the batch 39
Running for the batch 40
Running for the batch 41
Running 

In [ ]:
from collections import Counter

label_counts = Counter()

for _, labels in train_loader:
    labels = labels.cpu().numpy()  # move to CPU and convert to NumPy
    label_counts.update(labels.tolist())

print("Label distribution:")
for label, count in sorted(label_counts.items()):
    print(f"Label {label}: {count} samples")

In [ ]:
from collections import Counter

# Example for a custom dataset
label_counts = Counter(new_dataset_train.labels)  # or dataset.labels
print("Label distribution in Training:", label_counts)
# 0 = AI, 1 = Real

Label distribution: Counter({0: 105015, 1: 50000})


In [ ]:
from collections import Counter

# Example for a custom dataset
label_counts = Counter(new_dataset_val.labels)  # or dataset.labels
print("Label distribution in Validation:", label_counts)
# 0 = AI, 1 = Real

Label distribution: Counter({0: 20000, 1: 10000})
